In [3]:
DATA_PATH = '../data'
SHEET_NAME = None
TARGET = None
ID_COLS = []
FORCE_CATEGORICAL = []
FORCE_NUMERIC = []
SAMPLE_N = 1000
RANDOM_SEED = 42

In [5]:

import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

sns.set(style='whitegrid')
np.random.seed(RANDOM_SEED)

pd.options.display.max_columns = 200
pd.options.display.width = 120

# %%
# Funções utilitárias
def read_data(path, sheet_name=None):
    path = str(path)
    if path.endswith('.csv'):
        return pd.read_csv(path)
    if path.endswith(('.xls', '.xlsx')):
        return pd.read_excel(path, sheet_name=sheet_name)
    # fallback
    return pd.read_csv(path)

def set_dtypes(df):
    for c in FORCE_CATEGORICAL:
        if c in df.columns:
            df[c] = df[c].astype('category')
    for c in FORCE_NUMERIC:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')
    return df

def basic_report(df):
    print('Dimensão:', df.shape)
    display(df.head())
    print('\nTipos e não-nulos por coluna:')
    display(pd.DataFrame(df.dtypes, columns=['dtype']))
    print('\nContagem de valores nulos (colunas com >0 nulos):')
    display(df.isna().sum().loc[lambda x: x>0].sort_values(ascending=False))

def summary_statistics(df, numeric_only=True):
    if numeric_only:
        return df.describe().T
    else:
        return df.describe(include='all').T

def top_categories(df, col, n=10):
    return df[col].value_counts(dropna=False).head(n)

def sample_if_large(df, n=SAMPLE_N):
    return df.sample(n) if len(df)>n else df.copy()

def plot_hist_box(df, col, bins=30):
    fig, axes = plt.subplots(1,2, figsize=(12,4))
    sns.histplot(df[col].dropna(), bins=bins, ax=axes[0], kde=True)
    axes[0].set_title(f'Histograma: {col}')
    sns.boxplot(x=df[col].dropna(), ax=axes[1])
    axes[1].set_title(f'Boxplot: {col}')
    plt.tight_layout()
    plt.show()

def plot_count(df, col):
    plt.figure(figsize=(8,4))
    sns.countplot(y=col, data=df, order=df[col].value_counts().index)
    plt.title(f'Contagem por categoria: {col}')
    plt.show()

def corr_matrix(df, numeric_only=True):
    if numeric_only:
        num = df.select_dtypes(include=[np.number])
    else:
        num = df.copy()
    corr = num.corr()
    plt.figure(figsize=(10,8))
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', cbar_kws={'shrink':.8})
    plt.title('Matriz de correlação')
    plt.show()

def analyze_target(df, target):
    if target not in df.columns:
        print('Target não encontrada')
        return
    if pd.api.types.is_numeric_dtype(df[target]):
        print('Estatísticas da target numérica:')
        display(df[target].describe())
        plt.figure(figsize=(6,4))
        sns.histplot(df[target].dropna(), kde=True)
        plt.title('Distribuição da target')
        plt.show()
    else:
        print('Contagem por classe da target categórica:')
        display(df[target].value_counts())
        plt.figure(figsize=(6,4))
        sns.countplot(x=target, data=df)
        plt.title('Contagem da target')
        plt.show()

def detect_outliers_iqr(series):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - 1.5*iqr
    upper = q3 + 1.5*iqr
    return series[(series < lower) | (series > upper)]

# %%
# Carregar dados
df = read_data(DATA_PATH, sheet_name=SHEET_NAME)
df = set_dtypes(df)
print('Dados carregados com sucesso')

# %%
# Relatório básico
basic_report(df)

# %%
# Separar colunas numéricas e categóricas automaticamente
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = df.select_dtypes(include=['category','object','bool']).columns.tolist()
# remover id cols e target das listas se presentes
num_cols = [c for c in num_cols if c not in ID_COLS]
cat_cols = [c for c in cat_cols if c not in ID_COLS]
if TARGET:
    num_cols = [c for c in num_cols if c != TARGET]
    cat_cols = [c for c in cat_cols if c != TARGET]

print('Colunas numéricas:', num_cols)
print('Colunas categóricas:', cat_cols)

# %%
# Estatísticas descritivas numéricas
display(summary_statistics(df[num_cols]))

# Estatísticas para categóricas (top categories)
for c in cat_cols:
    print('\nColuna categórica:', c)
    display(top_categories(df, c, n=10))

# %%
# Valores faltantes
miss = df.isna().sum().sort_values(ascending=False)
miss = miss[miss>0]
display(miss)
if len(miss)>0:
    pct = (miss / len(df) * 100).round(2)
    display(pd.concat([miss, pct], axis=1, keys=['missing_count','missing_pct']))

# %%
# Visualizações univariadas automáticas (numéricas): histogramas e boxplots
for c in num_cols:
    try:
        print('\n==', c)
        plot_hist_box(df, c)
    except Exception as e:
        print('Erro ao plotar', c, e)

# %%
# Visualizações categóricas (top categories)
for c in cat_cols:
    try:
        print('\n==', c)
        plot_count(df, c)
    except Exception as e:
        print('Erro ao plotar', c, e)

# %%
# Matriz de correlação (entre numéricas)
if len(num_cols) > 1:
    corr_matrix(df[num_cols])
else:
    print('Poucas colunas numéricas para correlação')

# %%
# Análise da target (se informada)
if TARGET:
    analyze_target(df, TARGET)
else:
    print('TARGET não informada; defina TARGET na configuração para esta seção')

# %%
# Tratamento rápido de missing (exemplo) e codificação simples
def quick_impute_and_encode(df, numeric_strategy='median', fill_categorical='missing'):
    df_proc = df.copy()
    num = df_proc.select_dtypes(include=[np.number]).columns.tolist()
    cat = df_proc.select_dtypes(include=['object','category','bool']).columns.tolist()
    # Impute numéricos
    if num:
        imp_num = SimpleImputer(strategy=numeric_strategy)
        df_proc[num] = imp_num.fit_transform(df_proc[num])
    # Impute categóricos
    for c in cat:
        df_proc[c] = df_proc[c].fillna(fill_categorical)
        if df_proc[c].nunique() < 50:
            df_proc[c] = df_proc[c].astype('category')
    # One-hot para categóricas com poucas categorias
    small_cat = [c for c in cat if df_proc[c].nunique()<=10]
    if small_cat:
        df_proc = pd.get_dummies(df_proc, columns=small_cat, drop_first=True)
    return df_proc

df_proc = quick_impute_and_encode(df)
print('Dataset após imputação/encodificação rápida:', df_proc.shape)

# %%
# Detecção de outliers (IQR) em todas as numéricas
outliers = {c: detect_outliers_iqr(df[c].dropna()) for c in num_cols}
for c, series in outliers.items():
    if len(series)>0:
        print(f'Outliers detectados em {c}: {len(series)} valores')

# %%
# Split rápido em treino/teste (se target informada)
if TARGET and TARGET in df_proc.columns:
    X = df_proc.drop(columns=[TARGET]+ID_COLS if ID_COLS else [TARGET])
    y = df_proc[TARGET]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
    print('Tamanho treino/teste:', X_train.shape, X_test.shape)
else:
    print('Puleu split: TARGET não informada ou não encontrada no dataset processado')


IsADirectoryError: [Errno 21] Is a directory: '../data'